## 웹 크롤링 (Web Crawling)

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제 없음

In [9]:
from urllib.request import urlopen, Request #(Day5에 한것과 같음)

# 도시별 날씨 검색 함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text =  page.read().decode('utf-8')
    text = (text[text.find(f'>{city}</a>'):])

    # 기온 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]

    start = 3
    # td>를 없애기 위해 start가 3이 되어야함
    end = text.find('</td>')
    # 뒤에 있는 </td>를 삭제해줌
    current_temp = text[start:end]
    # 7.1이 나올 수 있게 해줌

    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

    # 앞에서 7까지 가져온 뒤이기 때문에 뒤에 3개가 필요함 
    # 10번째에 있는 내용을 가져와야하지만
    # => for i in range(10)안됨

    # 습도 가져오기
    for i in range(3):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]

    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')

get_weather('부산')

부산의 현재 기온은 11.6˚C 입니다.
부산의 현재 습도는 26% 입니다.


### OpenAPI 크롤링

- 공공데이터 포털
    - https://www.data.go.kr
    - 부산광역시 버스 정보 안내기 현황

```python
response = requests.get(total_url, verify=False)
```
- 문제점
    - https를 일반 request로 부르면 SSL 오류 발생
    - 해결하려면 외부모듈 requests 사용, verify=False 옵션 추가 지정해야함
    

In [38]:
# from urllib.request import urlopen, Request
import requests
from urllib.parse import quote, unquote, urlencode  
# quote : 한글로 된 글을 %~~ 이상한 글(url encode)로 변환해주는 함수
# 롯데 ->%EB%A1%AF%EB%8D%B0
import json
import ssl

def getrequesturl(url):
    req = Request(url)
    # Request 안써도 무방 
    # but, 사용시 조금 더 안전함

    try:
        res = urlopen(req)
        if res.getcode() == 200:
            return res.read().decode('utf-8')

    except Exception as e:
        print(e)
        return None

# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = '?' + urlencode(
        {
            'serviceKey' : '4Ng/JiC8rO2xbjXzST7yzTCSfv2+w57zIYoXivUfh1VIpBkQGln74kS5/yQpCMoANkXUJXlHWBEqNUrhHMo3zQ==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'resultType' : type,
            'stationLoc' : stationName
        }
    )
    total_url = api_url + queryString
    response = requests.get(total_url, verify=False)
    return response.text 

try:
    result = getDataPortalSearch('백화점', 'json')
    json_data = json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']
    for item in station_data:
        print(item)
except Exception as e:
    print('찾는 데이터가 없습니다.')


{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '05718', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15776525', 'lng': '129.0566948', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '05719', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15748116', 'lng': '129.0561774', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06030', 'stationLoc': '롯데백화점동래점.명륜역', 'lat': '35.21276314', 'lng': '129.0781263', 'addr': '동래구 온천1동 474-25', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06046', 'stationLoc': '롯데백화점동래점', 'lat': '35.2101971', 'lng': '129.0778307', 'addr': '동래구 온천2동 503-46', 'insYear': '', 'dataDay': '2023-02-03'}


c:\Source\StudyPython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

```python
stop_string = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
```
- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}'와 호환안됨
    - 구식방법인 문자열 결합방식으로 해결해야함

In [39]:
import folium

# 정류소 중 첫번째 인데스 정류소 위, 경도 중심으로 잡기
if len(station_data) >0 :
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']

m = folium.Map(location=[center_lat, center_lng], zoom_start=14)

# 전체 정류소 위치값 마커표시
for item in station_data:
    stop_string = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']

    iframe = folium.IFrame(stop_string)
    popup = folium.Popup(iframe, min_width = 200, max_width = 200) # popup에 popup 사이즈 설정하기

    folium.Marker(location=[item['lat'], item['lng']], popup=popup, 
                  icon=folium.Icon(icon='pushpin')).add_to(m)

m

### BeautifulSoup (version 4)

웹크롤링을 편하게 해주는 도구(라이브러리)

```shell
pip install beautifulsoup4
```

In [28]:
!pip install beautifulsoup4

     -------------------------------------- 129.4/129.4 kB 3.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 모듈 import

```python
from bs4 import beautifulsoup
```

In [35]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)

if response.status_code == 200: #웹페이지 OK 라는 의미
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title)
    print(title.get_text())
    # print(soup)
else:
    print(f'Error : {response.status_code}')

<a class="_nclicks:kin.txt _searchListTitleAnchor" href="https://kin.naver.com/qna/detail.naver?d1id=11&amp;dirId=1129&amp;docId=423747412&amp;qb=7KCc7J6E7IqkIOybuSDrp53sm5Dqsr0=&amp;enc=utf8§ion=kin&amp;rank=1&amp;search_sort=0&amp;spq=0" target="_blank"><b>제임스웹</b> 우주<b>망원경</b> 질문이요</a>
제임스웹 우주망원경 질문이요


In [37]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)

if response.status_code == 200: #웹페이지 OK 라는 의미
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error : {response.status_code}')

제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
